[View in Colaboratory](https://colab.research.google.com/github/richard-cartwright/personal/blob/master/PowerCo_churn.ipynb)

# SUMMARY

This is for a consulting case study. The context is that I am working with an imaginary power company. They are concerned by higher-than-industry churn, and are interested in using their customer data to predict which customers will churn.

This is meant as a first exploratory meeting with the client, so they are interested in the feasibility of a predictive model. My task is therefore to first explore the data, test the feasibility of a model, and create a first attempt at a model.

My presentation slides are here:
https://docs.google.com/presentation/d/1jsCm4DJdLRK5zYzfL2Fk4cds_YRX_zDoqrJt6XbOMyE/edit?usp=sharing

## Story

1) The data presented is messy. I therefore spend much time 'sharpening my axe': smartly imputing missing data, creating dummy variables etc. I question the accuracy of the data itself

2) This work is exploratory. I therefore do some basic data visualisation, highlighting the most important features for prediction.

3) While modelling, I always start with out-of-the-box basic prediction to set a baseline. I then scale my data, interact features to create more complex features, then use feature selection to reduce the size of my input data to make my models run quicker.

4) I tune hyperparameters for KNN, then do the same for Random Forest. I have to deal with class imbalance by upsampling as only ~10% customers churn.

5) I test my tuned model using my hold-out test data to set an expectation of accuracy based on unseen data. Finally, I train a model (with tuned hyperparameters) using all labelled data, and use this model on the unlabelled submission data.

## CONTENTS






In [0]:
# 1) PACKAGES
#   a) Installs
#   b) Imports

# 2) ENVIRON SET-UP

# 3) DATA CLEANING & FEATURE ENGINEERING
#   a) Merge Train & Test
#   b) Model Data
#   c) 'Date' Features
#   d) 'Forecast' Features
#   e) 'Cons' Features
#   f) Ordinal Features
#   g) Categorical Features
#   h) Other Features
#   i) Separating into Model & Submission
  
# 4) DATA VIZ

# 5) MODELLING
#   a) Train-Val-Test Split
#   b) Basic Models
#   c) Feature Scaling & Selection
#   d) K-Nearest Neighbors
#   e) Random Forest
#   f) Class Imbalance
  
# 6) SUBMISSION

# PACKAGES

## Installs

In [0]:
# Install to visualise ROC curve
!pip install scikit-plot

## Imports

In [0]:
# Basic imports, including ML libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pprint
%matplotlib inline

# Setting plotting styles
plt.style.use('fivethirtyeight')
sns.set_style('white')

# Displays all cell's output, not just last output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Sklearn
import scikitplot as skplt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, recall_score

# Tensorflow & Keras
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# ENVIRON SET-UP

In [0]:
# Add GDrive to Colab environment

from google.colab import drive
drive.mount('/content/drive')

# Create path for data
path = '/content/drive/My Drive/Colab Notebooks/Personal/BCG Gamma/ml_case_data'

In [0]:
# Extract data

# Training data
training_data = pd.read_csv(path+'/training_data.csv', 
                        index_col='id', 
                        parse_dates=['date_activ',
                                     'date_end', 
                                     'date_first_activ',
                                     'date_modif_prod', 
                                     'date_renewal'])
training_hist_data = pd.read_csv(path+'/training_hist_data.csv',
                             index_col=['id','price_date'], 
                             parse_dates=['price_date'])
training_output = pd.read_csv(path+'/training_output.csv', 
                              index_col='id')

# Submission data
test_data = pd.read_csv(path+'/test_data.csv', 
                        index_col='id', 
                        parse_dates=['date_activ',
                                     'date_end', 
                                     'date_first_activ',
                                     'date_modif_prod', 
                                     'date_renewal'])
test_hist_data = pd.read_csv(path+'/test_hist_data.csv',
                             index_col=['id','price_date'], 
                             parse_dates=['price_date'])

# DATA CLEANING & FEATURE ENGINEERING

## Merge Train & Test

In [0]:
# Merge model & submission datasets so they go through the same data preprocessing

# Create labels for model & submission datasets
training_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

# Concat model & submission datasets
whole_data = pd.concat([training_data,test_data])
whole_hist_data = pd.concat([training_hist_data,test_hist_data])

## Model Data

In [0]:
# Price data for each customer(level 0), for each month of 2015 (level 1)
# There are six separate price measures (features)

# Calculates proportion of missing data for each customer for each of the six price measures
missing_prices = whole_hist_data.isnull().groupby(level=0).mean()
missing_prices.columns = ['missing_'+col for col in missing_prices.columns]

# Calculates mean for each of the six price measures for each customer
mean_prices = whole_hist_data.groupby(level=0).mean()
mean_prices.columns = ['mean_'+col for col in mean_prices.columns]

# Calculates range (max-min) for each of the six price measures for each customer
# Captures whether customer has seen a price change
range_prices = whole_hist_data.groupby(level=0).max() - whole_hist_data.groupby(level=0).min()
range_prices.columns = ['range_'+col for col in range_prices.columns]

# Calculates change (last-first) for each of the six price measures for each customer
# Captures size & direction of price change for a customer
change_prices = whole_hist_data.groupby(level=0).last() - whole_hist_data.groupby(level=0).first()
change_prices.columns = ['change_'+col for col in change_prices.columns]

# Concat to create df of all price features
whole_prices = pd.concat([missing_prices,mean_prices,range_prices,change_prices], axis=1)

# Use median to fillnas
whole_prices = whole_prices.fillna(whole_prices.median())

In [0]:
# Set up model data & target variable

# Concat indiv data & price data
model_data = pd.concat([whole_data, whole_prices], axis=1).sort_index()

# Churn (True or False) is the target variable
target = training_output['churn'].sort_index()

model_data.info()

# Zero data points for 'campaign_disc_ele', therefore drop
model_data.drop('campaign_disc_ele', axis=1, inplace=True)

## 'Date' Features

In [0]:
# Extract 'date' columns

date_cols = [col for col in model_data.columns if col[:4]=='date']

model_data[date_cols].info()
model_data[date_cols].head(10)

In [0]:
# 'date_end'
# Only two missing, and no real pattern - therefore make missing values the median

feature = 'date_end'
print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))

# Create bool column if feature is missing 
model_data['missing_'+feature] = model_data[feature].isnull()

# Fill missing values with feature median
median_date_end = sorted(model_data[feature])[len(model_data[feature])//2]
model_data[feature] = model_data[feature].fillna(median_date_end)

In [0]:
# 'date_renewal'
# date_renewal always exactly one year before date_end (more precisely, from 364-362 days before)

feature = 'date_renewal'
print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))

# Create bool column if feature is missing 
model_data['missing_'+feature] = model_data[feature].isnull()

# Fill missing values with 'date_end' minus 364 days
model_data[feature] = model_data[feature].fillna(model_data['date_end'] - pd.DateOffset(years=1,days=-1))

In [0]:
# 'date_modif_prod'
# For the vast majority of 'date_modif_prod', it is exactly the same as 'date_activ'
# Therefore, set missing 'date_modif_prod'='date_activ'

feature = 'date_modif_prod'
print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))

# Create bool column if feature is missing 
model_data['missing_'+feature] = model_data[feature].isnull()

# Fill missing values with 'date_activ'
model_data[feature] = model_data[feature].fillna(model_data['date_activ'])

In [0]:
# date_first_activ
# For the non-missing 'date_first_activ', they are exactly the same as 'date_activ'
# Therefore, set missing 'date_first_activ'='date_activ'

feature = 'date_first_activ'
print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))

# Create bool column if feature is missing 
model_data['missing_'+feature] = model_data[feature].isnull()

# Fill missing values with 'date_activ'
model_data[feature] = model_data[feature].fillna(model_data['date_activ'])

In [0]:
# For each of the five date columns, extract 'year' and 'month'
# Year is ordinal so leave as one feature, whereas Quarter is categorical so create dummy vars

quarter_date_cols = []
# For each date col, create Year & Quarter features
for col in date_cols:
  model_data['quarter_'+col] = model_data[col].dt.quarter
  model_data['year_'+col] = model_data[col].dt.year
  quarter_date_cols.extend(['quarter_'+col])

# Drop original datetime cols
model_data = model_data.drop(date_cols,axis=1)

# Create dummy variables for Quarter features
model_data = pd.get_dummies(model_data, columns=quarter_date_cols, drop_first=True)

## 'Forecast' Features

In [0]:
# Extract 'forecast' columns

forecast_cols = [col for col in model_data.columns if col[:8]=='forecast']

model_data[forecast_cols].info()
model_data[forecast_cols].head(10)

In [0]:
# Loads of these columns have excessive zeros - could be a data inaccuracy.
# Therefore, create extra bool feature if value = zero, and then set value to feature median. 

for col in forecast_cols:
  feature = col
  print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))
  print('#zeros out of 20120 for {}:'.format(feature),sum((model_data[feature]==0)))

  # Create bool column if feature is missing or zero
  model_data['missing_'+feature] = model_data[feature].isnull()
  model_data['zero_'+feature] = (model_data[feature]==0)
  
  # Replace missings and zeros with feature median
  col_median = model_data[feature].median()
  model_data[feature] = model_data[feature].replace(to_replace=0, value=col_median)
  model_data[feature] = model_data[feature].fillna(col_median)

## 'Cons' Features

In [0]:
# Extract 'cons' columns

cons_cols = [col for col in model_data.columns if col[:4]=='cons']

model_data[cons_cols].info()
model_data[cons_cols].head(10)

In [0]:
# Loads of these columns have excessive zeros - could be a data inaccuracy.
# Therefore, create extra bool feature if value = zero, and then set value to feature median. 

for col in cons_cols:
  feature = col
  print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))
  print('#zeros out of 20120 for {}:'.format(feature),sum((model_data[feature]==0)))

  # Create bool column if feature is missing or zero
  model_data['missing_'+feature] = model_data[feature].isnull()
  model_data['zero_'+feature] = (model_data[feature]==0)
  
  # Replace missings and zeros with feature median
  col_median = model_data[feature].median()
  model_data[feature] = model_data[feature].replace(to_replace=0, value=col_median)
  model_data[feature] = model_data[feature].fillna(col_median)

## Ordinal Features

In [0]:
# Extract Ordinal columns

ordinal_cols = ['nb_prod_act', 'num_years_antig']

model_data[ordinal_cols].info()
model_data[ordinal_cols].head(10)

In [0]:
# Planned to restrict the range of the data so I could make dummy variables.
# But in the end left as Ordinal variables with data untouched.

In [0]:
# 'nb_prod_act'

# model_data['nb_prod_act'].groupby(model_data['nb_prod_act']).count()
# model_data.loc[model_data['nb_prod_act'] >=4, 'nb_prod_act'] = 4

In [0]:
# 'num_years_antig'

# model_data['num_years_antig'].groupby(model_data['num_years_antig']).count()
# model_data.loc[model_data['num_years_antig'] <=3, 'num_years_antig'] = 3
# model_data.loc[model_data['num_years_antig'] >=12, 'num_years_antig'] = 12

## Categorical Features

In [0]:
# Extract Categorical columns

categorical_cols = ['activity_new', 'channel_sales', 'origin_up']

model_data[categorical_cols].info()
model_data[categorical_cols].head(10)

In [0]:
# If data is missing for a feature, fill with 'missing'
# If the data is part of a category with <=100 hits, replace with 'too_low'.
# Do this so have a reasonable (lower) amount of dummy variables.

for col in categorical_cols:
  feature = col

  # Fill missing with 'missing'
  model_data[feature] = model_data[feature].fillna('missing')

  # Count of hits for each category for the feature
  col_distn = model_data[feature].groupby(model_data[feature]).count().sort_values(ascending=False)
  
  # Replace categories with <=100 hits with 'too_low' - reduces the amount of dummy vars
  feature_replace = list(set(col_distn.index) - set(col_distn[col_distn.where(col_distn>100).notnull()].index))
  model_data[feature] = model_data[feature].replace(to_replace=feature_replace, value='too_low')
  
  # Cuts feature name from long hash to first 7 characters
  model_data[feature] = model_data[feature].apply(lambda x: x[:7])
  
# Create dummy vars for the categorical vars
model_data = pd.get_dummies(model_data, columns=categorical_cols, drop_first=True)

In [0]:
# 'has_gas'
# Originally, 't' is True and 'f' is False. This turns it into a bool column.

model_data['has_gas'] = (model_data['has_gas'] == 't')

## Other Features

In [0]:
# Extract remaining (numerical) columns

other_cols = ['imp_cons',
              'margin_gross_pow_ele',
              'margin_net_pow_ele',
              'net_margin',
              'pow_max']

model_data[other_cols].info()
model_data[other_cols].head(10)

In [0]:
# Loads of these columns have excessive zeros - could be a data inaccuracy.
# Therefore, create extra bool feature if value = zero, and then set value to feature median. 

for col in other_cols:
  feature = col
  print('#nulls out of 20120 for {}:'.format(feature),sum(model_data[feature].isnull()))
  print('#zeros out of 20120 for {}:'.format(feature),sum((model_data[feature]==0)))

   # Create bool column if feature is missing or zero
  model_data['missing_'+feature] = model_data[feature].isnull()
  model_data['zero_'+feature] = (model_data[feature]==0)
  
  # Replace missings and zeros with feature median
  col_median = model_data[feature].median()
  model_data[feature] = model_data[feature].replace(to_replace=0, value=col_median)
  model_data[feature] = model_data[feature].fillna(col_median)

## Separating into Model & Submission

In [0]:
# Reseparate data into Model & Submission data

model_data_train = model_data[model_data.train_or_test == 'train'].drop('train_or_test',axis=1)
model_data_submission = model_data[model_data.train_or_test == 'test'].drop('train_or_test',axis=1)

# DATA VIZ

In [0]:
# Get initial feature importances for visualisation

# First scale data as better for viz
scaled_data = pd.DataFrame(data=StandardScaler().fit_transform(model_data_train), 
                           index=model_data_train.index, 
                           columns=model_data_train.columns)

# Use out-of-box RandomForest for feature importances
rfc = RandomForestClassifier()
RandomForestClassifier().fit(scaled_data, target)
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = scaled_data.columns,
                                   columns=['importance']).sort_values('importance',ascending=False)

# Print 10 most important features
print(feature_importances.head(10),'\n')

# Extract & visualise via boxplots the 10 most important features
features = list(feature_importances.index)[:10]
plt.figure(figsize=(12,6))
sns.boxplot(scaled_data[features], 
            showfliers=False, 
            vert=False);

In [0]:
# Usa PCA to visualise data in 2D

pca_2d = PCA(n_components=2,random_state=42)
twodim_pca_data = pd.DataFrame(data=pca_2d.fit_transform(scaled_data),
                               index=scaled_data.index,
                               columns=['x1','x2'])

# Plot 2 dimensions, with whether 'churn'==1 dictating the point's colour
two_dim_viz_df = pd.concat([twodim_pca_data,target], axis=1)
two_dim_viz_df.plot.scatter('x1','x2',c='churn',alpha=0.2,cmap='RdYlGn');

In [0]:
# Visualising difference in churn based on when the customer first became active

# For each year, get mean proportion who churned
viz_data = pd.concat([model_data_train,target], axis=1).groupby('year_date_first_activ')['churn'].mean()

# Visualise only 2008 to 2013
viz_data = viz_data.loc[2008:2013]
viz_data.plot(kind='bar',
              title='Proportion churn by year of date of first contract',
              rot=0);
plt.ylabel('Proportion who churn \n in a given year');

# MODELLING

## Train-Val-Test Split

In [0]:
# Split data into Train-Val-Test (60-20-20)

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(model_data_train, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=42)

# withold 25% of training set for validation of hyperparameters
validation_threshold = round(len(X_train)*3/4)
X_val = X_train.iloc[validation_threshold:]
y_val = y_train[validation_threshold:]

# make X_train & y_train exclude validation data
X_train = X_train.iloc[:validation_threshold]
y_train = y_train[:validation_threshold]

## Basic Models

In [0]:
# LogReg out-of-the-box to get a baseline - plot ROCs for training & test

# no optimisation, just to see basic accuracy
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train,y_train)

probs_train = logreg.predict_proba(X_train)
probs_test = logreg.predict_proba(X_test)

print('Basic logreg training roc_auc_score:', roc_auc_score(y_train,probs_train[:,1]))
print('Basic logreg test roc_auc_score:', roc_auc_score(y_test,probs_test[:,1]))

# Plot ROCs for Train & Test
skplt.metrics.plot_roc(y_train,probs_train, title='LogReg Train ROC');
skplt.metrics.plot_roc(y_test,probs_test, title='LogReg Test ROC');

In [0]:
# RandomForest out-of-the-box to get a baseline - plot ROCs for training & test

# no optimisation, just to see basic accuracy
forest = RandomForestClassifier(random_state=42,
                                n_estimators=100,
                                max_depth=10)
forest.fit(X_train,y_train)

probs_train = forest.predict_proba(X_train)
probs_test = forest.predict_proba(X_test)

print('Basic RandomForest training roc_auc_score:', roc_auc_score(y_train,probs_train[:,1]))
print('Basic RandomForest test roc_auc_score:', roc_auc_score(y_test,probs_test[:,1]))

# Plot ROCs for Train & Test
skplt.metrics.plot_roc(y_train,probs_train, title='RandomForest Train ROC');
skplt.metrics.plot_roc(y_test,probs_test, title='RandomForest Test ROC');

## Feature Scaling & Selection

In [0]:
# Create pipeline for Feature Scaling & Selection
# First scale features,
# then create interactions to poly=2, 
# then use PCA to reduce to 50 features

features_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('polyFeat', PolynomialFeatures(degree=2, 
                                    include_bias=False)),
    ('pca', PCA(n_components=50, 
                random_state=42))
])

# Fit then transform Train-Val-Test data
X_train = features_pipe.fit_transform(X_train)
X_val = features_pipe.transform(X_val)
X_test = features_pipe.transform(X_test)

## K-Nearest Neighbors

In [0]:
# Hyperparameter tuning for KNN

# Implementation takes a few minutes, so commented out to avoid accidentally running.

# neighbors_params = [1,2,4,7,11,16,22,29,37,46,56,67]
# knn_train_scores = []
# knn_val_scores = []

# for k in neighbors_params:
#   knn = KNeighborsClassifier(n_neighbors=k)
#   knn.fit(X_train,y_train)
#   # print('\n')
  
#   probs_train = knn.predict_proba(X_train)
#   probs_val = knn.predict_proba(X_val)
  
#   knn_train_scores.append(roc_auc_score(y_train,probs_train[:,1]))
#   knn_val_scores.append(roc_auc_score(y_val,probs_val[:,1]))

In [0]:
# Plot Train & Val scores for different K in KNN

knn_scores_df = pd.DataFrame(data={'train':knn_train_scores,
                                   'val':knn_val_scores},
                             index=neighbors_params)
knn_scores_df.plot(title='KNN roc_auc_score for different k');
plt.xlabel('#neighbors');
plt.ylabel('roc_auc_score');

## Random Forest

In [0]:
# Hyperparameter tuning for Random Forest, using the two key hyperparameters:
# i) Number of estimators: how many trees in forest
# ii) Maximum depth: maximum layers of nodes

# Implementation takes a few minutes, so commented out to avoid accidentally running.

# tree_estimators_params = [int(round(n)) for n in np.logspace(0,3,num=4)]
# tree_depth_params = [int(round(depth)) for depth in np.logspace(0.5,2,num=4)]
# tree_scores_df = pd.DataFrame(columns=['n_estimators',
#                                        'max_depth',
#                                        'train_score',
#                                        'val_score'])

# for n in tree_estimators_params:
#   for depth in tree_depth_params:
#     forest = RandomForestClassifier(random_state=42,
#                                     n_estimators=n,
#                                     max_depth=depth)
#     forest.fit(X_train,y_train)

#     probs_train = forest.predict_proba(X_train)
#     probs_val = forest.predict_proba(X_val)
    
#     tree_scores_df = tree_scores_df.append(
#       {'n_estimators':n,
#        'max_depth':depth,
#        'train_score':roc_auc_score(y_train,probs_train[:,1]),
#        'val_score':roc_auc_score(y_val,probs_val[:,1])
#       },
#       ignore_index=True
#     )

In [0]:
# Create heatmap of n_estimators vs max_depth, with heat=validation_score

sns.heatmap(tree_scores_df.pivot_table(values='val_score',index='n_estimators',columns='max_depth'),
            annot=True);
plt.title('Validation score for two key hyperparams \n for Random Forest');

## Class Imbalance

In [0]:
# Only ~10% of customers churn - therefore problem of class imbalance.
# Recall is very low. This means that of those who do actually churn, very few are predicted to do so.

# Therefore, I try out different reweightings for churn==1 class, from 1 to 144.

# Implementation takes a few minutes, so commented out to avoid accidentally running.

# tree_weighting_params = [i**2 for i in range(1,13)]
# tree_weighting_scores_df = pd.DataFrame(columns=['weighting',
#                                                  'train_score',
#                                                  'val_score',
#                                                  'val_recall'])

# for weight in tree_weighting_params:
#   forest = RandomForestClassifier(random_state=42,
#                                   n_estimators=100,
#                                   max_depth=32,
#                                   class_weight={1:weight})
#   forest.fit(X_train,y_train)

#   probs_train = forest.predict_proba(X_train)
#   probs_val = forest.predict_proba(X_val)

#   tree_weighting_scores_df = tree_weighting_scores_df.append(
#     {'weighting':weight,
#      'train_score':roc_auc_score(y_train,probs_train[:,1]),
#      'val_score':roc_auc_score(y_val,probs_val[:,1]),
#      'val_recall':recall_score(y_val,forest.predict(X_val))
#     },
#     ignore_index=True
#   )

In [0]:
# Plot the train_score, val_score, and val_recall against the different weightings
tree_weighting_scores_df.set_index('weighting').plot(title='Upsampling the churn=1 data to combat \n class imbalance');
plt.xlabel('Weighting on churn=1 class');

# SUBMISSION

In [0]:
# Use optimal hyperparameters to fit model, extracting final auc_score on Test data.

forest = RandomForestClassifier(random_state=42,
                                n_estimators=100,
                                max_depth=32,
                                class_weight={1:5})

forest.fit(X_train,y_train);
probs_test = forest.predict_proba(X_test)
print('\n \n', 'Final test roc_auc_score:', roc_auc_score(y_test,probs_test[:,1]))

In [0]:
# Use Random Forest as the model.
# Use optimal hyperparameters and entire model dataset (Train-Val-Test) to fit model.
# Make predictions (with probabilities) for submission  data.

# Model data uses all of labelled data, submission data is without labels
X_model = features_pipe.fit_transform(model_data_train)
X_submission = features_pipe.transform(model_data_submission)

# Random Forest with optimised hyperparameters
forest = RandomForestClassifier(random_state=42,
                                n_estimators=100,
                                max_depth=32,
                                class_weight={1:5})
forest.fit(X_model,target)

# Use fitted model to predict labels and probabilities for submission data
submission_df = pd.DataFrame(
    data={'Churn_prediction':forest.predict(X_submission),
          'Churn_probability':forest.predict_proba(X_submission)[:,1]
         },
    index=model_data_submission.index
)

In [0]:
# Download submission as a csv

# submission_df.to_csv('Powerco_submission_RichardCartwright.csv', index=True)
# from google.colab import files
# files.download('Powerco_submission_RichardCartwright.csv')